# Rakenna regressiomalli: lineaariset ja polynomiset regressiomallit


## Lineaarinen ja polynominen regressio kurpitsan hinnoitteluun - Oppitunti 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografiikka: Dasani Madipalli</figcaption>


<!--![Infografiikka: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Johdanto

Tähän mennessä olet tutustunut siihen, mitä regressio tarkoittaa, käyttäen esimerkkidataa kurpitsan hinnoitteludatasetistä, jota käytämme koko tämän oppitunnin ajan. Olet myös visualisoinut sitä `ggplot2`:n avulla.💪

Nyt olet valmis sukeltamaan syvemmälle regressioon koneoppimisen näkökulmasta. Tässä oppitunnissa opit lisää kahdesta regressiotyypistä: *yksinkertainen lineaarinen regressio* ja *polynominen regressio*, sekä hieman näiden tekniikoiden taustalla olevasta matematiikasta.

> Koko tämän kurssin ajan oletamme vain vähäistä matematiikan osaamista ja pyrimme tekemään sisällön helposti lähestyttäväksi opiskelijoille, jotka tulevat muilta aloilta. Huomioi siis muistiinpanot, 🧮 matemaattiset esimerkit, kaaviot ja muut oppimistyökalut, jotka auttavat ymmärtämisessä.

#### Valmistelu

Muistutuksena, lataat tätä dataa, jotta voit esittää kysymyksiä siitä.

-   Milloin on paras aika ostaa kurpitsoja?

-   Minkä hinnan voin odottaa minikurpitsojen laatikolle?

-   Pitäisikö minun ostaa niitä puolibushelin koreissa vai 1 1/9 bushelin laatikoissa? Jatketaan datan tutkimista.

Edellisessä oppitunnissa loit `tibble`:n (nykyaikainen uudelleentulkinta dataframesta) ja täytit sen osalla alkuperäistä datasettiä, standardisoiden hinnoittelun bushelin mukaan. Tällä tavalla pystyit kuitenkin keräämään vain noin 400 datapistettä ja vain syksyn kuukausilta. Ehkä voimme saada hieman enemmän yksityiskohtia datan luonteesta puhdistamalla sitä lisää? Katsotaan... 🕵️‍♀️

Tätä tehtävää varten tarvitsemme seuraavat paketit:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [R-pakettien kokoelma](https://www.tidyverse.org/packages), joka on suunniteltu tekemään datatiede nopeammaksi, helpommaksi ja hauskemmaksi!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) -kehys on [pakettikokoelma](https://www.tidymodels.org/packages/) mallintamiseen ja koneoppimiseen.

-   `janitor`: [janitor-paketti](https://github.com/sfirke/janitor) tarjoaa yksinkertaisia työkaluja likaisen datan tarkasteluun ja puhdistamiseen.

-   `corrplot`: [corrplot-paketti](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) tarjoaa visuaalisen tutkimustyökalun korrelaatiomatriisille, joka tukee automaattista muuttujien järjestämistä piilotettujen kuvioiden havaitsemiseksi muuttujien välillä.

Voit asentaa ne seuraavasti:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Alla oleva skripti tarkistaa, onko sinulla tarvittavat paketit tämän moduulin suorittamiseen, ja asentaa ne puolestasi, jos ne puuttuvat.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Lataamme myöhemmin nämä upeat paketit ja teemme ne saatavilla nykyisessä R-istunnossamme. (Tämä on vain havainnollistamista varten, `pacman::p_load()` teki tämän jo puolestasi)

## 1. Lineaarinen regressioviiva

Kuten opit oppitunnilla 1, lineaarisen regressioharjoituksen tavoitteena on pystyä piirtämään *paras sovitusviiva*:

-   **Näytä muuttujien väliset suhteet**. Näytä muuttujien välinen suhde.

-   **Tee ennusteita**. Tee tarkkoja ennusteita siitä, mihin uusi datapiste sijoittuisi suhteessa tähän viivaan.

Tämän tyyppisen viivan piirtämiseen käytämme tilastollista menetelmää nimeltä **Pienimmän neliösumman regressio**. Termi `pienimmän neliösumman` tarkoittaa, että kaikki regressioviivan ympärillä olevat datapisteet neliöidään ja sitten summataan. Ihanteellisesti tämä lopullinen summa on mahdollisimman pieni, koska haluamme vähän virheitä eli `pienimmän neliösumman`. Näin ollen paras sovitusviiva on viiva, joka antaa meille pienimmän arvon neliöityjen virheiden summalle - tästä nimi *pienimmän neliösumman regressio*.

Teemme tämän, koska haluamme mallintaa viivan, jolla on pienin kumulatiivinen etäisyys kaikista datapisteistämme. Neliöimme myös termit ennen niiden yhteenlaskemista, koska olemme kiinnostuneita niiden suuruudesta, emmekä suunnasta.

> **🧮 Näytä matematiikka**
>
> Tämä viiva, nimeltään *paras sovitusviiva*, voidaan ilmaista [yhtälöllä](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` on '`selittävä muuttuja` tai `ennustaja`'. `Y` on '`riippuva muuttuja` tai `tulos`'. Viivan kulmakerroin on `b` ja `a` on y-akselin leikkauspiste, joka viittaa `Y`:n arvoon, kun `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "kulmakerroin = $y/x$")
    Infografiikka: Jen Looper
>
> Ensin lasketaan kulmakerroin `b`.
>
> Toisin sanoen, viitaten kurpitsadatan alkuperäiseen kysymykseen: "ennusta kurpitsan hinta per bushel kuukauden mukaan", `X` viittaa hintaan ja `Y` viittaa myyntikuukauteen.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografiikka: Jen Looper
> 
> Laske Y:n arvo. Jos maksat noin 4 dollaria, sen täytyy olla huhtikuu!
>
> Matematiikka, joka laskee viivan, täytyy osoittaa viivan kulmakerroin, joka riippuu myös leikkauspisteestä, eli siitä, missä `Y` sijaitsee, kun `X = 0`.
>
> Voit tarkastella näiden arvojen laskentamenetelmää [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) -sivustolla. Käy myös [tällä pienimmän neliösumman laskimella](https://www.mathsisfun.com/data/least-squares-calculator.html) nähdäksesi, miten lukuarvot vaikuttavat viivaan.

Ei niin pelottavaa, eikö? 🤓

#### Korrelaatio

Yksi termi, joka on hyvä ymmärtää, on **korrelaatiokerroin** annettujen X- ja Y-muuttujien välillä. Hajontakaavion avulla voit nopeasti visualisoida tämän kertoimen. Kaavio, jossa datapisteet ovat siistissä linjassa, omaa korkean korrelaation, mutta kaavio, jossa datapisteet ovat hajallaan X:n ja Y:n välillä, omaa matalan korrelaation.

Hyvä lineaarinen regressiomalli on sellainen, jolla on korkea (lähempänä 1 kuin 0) korrelaatiokerroin käyttäen pienimmän neliösumman regressiomenetelmää ja regressioviivaa.


## **2. Tanssi datan kanssa: luodaan data frame mallinnusta varten**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Taiteilija: @allison_horst</figcaption>


<!--![Taiteilija: \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Lataa tarvittavat kirjastot ja datasetti. Muunna data data frameksi, joka sisältää osajoukon datasta:

-   Valitse vain kurpitsat, joiden hinta on ilmoitettu tynnyrillisinä

-   Muunna päivämäärä kuukaudeksi

-   Laske hinta korkeiden ja matalien hintojen keskiarvona

-   Muunna hinta vastaamaan tynnyrillistä kohden laskettua hinnoittelua

> Kävimme nämä vaiheet läpi [edellisessä oppitunnissa](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Seikkailun hengessä tutustutaan [`janitor-pakettiin`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), joka tarjoaa yksinkertaisia funktioita likaisen datan tarkasteluun ja puhdistamiseen. Esimerkiksi tarkastellaan datamme sarakenimiä:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Voimme tehdä paremmin. Muutetaan nämä sarakenimet `friendR` käyttämällä [snake_case](https://en.wikipedia.org/wiki/Snake_case) -konventiota `janitor::clean_names` avulla. Lisätietoja tästä funktiosta: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Paljon siistimpää 🧹! Nyt tanssia datan kanssa käyttäen `dplyr` kuten edellisessä oppitunnissa! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Hyvää työtä!👌 Sinulla on nyt siisti ja järjestetty datajoukko, jonka pohjalta voit rakentaa uuden regressiomallisi!

Miltä kuulostaisi hajontakaavio?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Hajontakaavio muistuttaa meitä siitä, että meillä on kuukausitietoja vain elokuusta joulukuuhun. Tarvitsemme luultavasti lisää tietoa voidaksemme tehdä johtopäätöksiä lineaarisesti.

Katsotaanpa mallinnustietojamme uudelleen:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Entä jos haluaisimme ennustaa kurpitsan `hinnan` perustuen `kaupunki`- tai `paketti`-sarakkeisiin, jotka ovat merkkijonotyyppisiä? Tai vielä yksinkertaisemmin, miten voisimme löytää korrelaation (joka vaatii molempien syötteiden olevan numeerisia) esimerkiksi `paketti`- ja `hinta`-sarakkeiden välillä? 🤷🤷

Koneoppimismallit toimivat parhaiten numeeristen ominaisuuksien kanssa tekstiarvojen sijaan, joten yleensä täytyy muuntaa kategoriset ominaisuudet numeerisiksi esityksiksi.

Tämä tarkoittaa, että meidän täytyy löytää tapa muokata ennustettavia muuttujia, jotta malli voi käyttää niitä tehokkaasti. Tätä prosessia kutsutaan nimellä `ominaisuuksien suunnittelu` (feature engineering).


## 3. Datan esikäsittely mallinnusta varten reseptien avulla 👩‍🍳👨‍🍳

Toiminnot, jotka muokkaavat ennustearvoja, jotta malli voi käyttää niitä tehokkaammin, tunnetaan nimellä `ominaisuuksien muokkaus`.

Eri malleilla on erilaiset esikäsittelyvaatimukset. Esimerkiksi pienimmän neliösumman menetelmä vaatii `kategoristen muuttujien koodaamista`, kuten kuukausi, lajike ja city_name. Tämä tarkoittaa yksinkertaisesti `kategoristen arvojen sisältävän sarakkeen` muuttamista yhdeksi tai useammaksi `numeeriseksi sarakkeeksi`, jotka korvaavat alkuperäisen.

Esimerkiksi, jos datasi sisältää seuraavan kategorisen ominaisuuden:

|  kaupunki   |
|:-----------:|
| Denver      |
| Nairobi     |
|  Tokio      |

Voit käyttää *järjestyskooodausta* korvataksesi jokaisen kategorian ainutlaatuisella kokonaisluvulla, kuten näin:

| kaupunki |
|:--------:|
|    0     |
|    1     |
|    2     |

Ja juuri näin teemme datallemme!

Tässä osiossa tutustumme toiseen upeaan Tidymodels-pakettiin: [recipes](https://tidymodels.github.io/recipes/) - joka on suunniteltu auttamaan datan esikäsittelyssä **ennen** mallin kouluttamista. Resepti on pohjimmiltaan objekti, joka määrittelee, mitä vaiheita dataan tulee soveltaa, jotta se olisi valmis mallinnusta varten.

Nyt luodaan resepti, joka valmistaa datamme mallinnusta varten korvaamalla ainutlaatuisen kokonaisluvun kaikille ennustearvojen sarakkeiden havainnoille:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Mahtavaa! 👏 Juuri loimme ensimmäisen reseptimme, joka määrittää lopputuloksen (hinta) ja siihen liittyvät ennustajat, ja että kaikki ennustajakolumnit tulisi koodata kokonaislukujen joukkoon 🙌! Puretaanpa tämä nopeasti:

-   Kutsu `recipe()`-funktiolle kaavan kanssa kertoo reseptille muuttujien *roolit* käyttäen `new_pumpkins`-dataa viitteenä. Esimerkiksi `price`-kolumnille on annettu `outcome`-rooli, kun taas lopuille kolumneille on annettu `predictor`-rooli.

-   `step_integer(all_predictors(), zero_based = TRUE)` määrittää, että kaikki ennustajat tulisi muuntaa kokonaislukujen joukkoon, jossa numerointi alkaa nollasta.

Olemme varmoja, että sinulla saattaa olla ajatuksia, kuten: "Tämä on niin siistiä!! Mutta entä jos minun pitäisi varmistaa, että reseptit tekevät juuri sen, mitä odotan niiden tekevän? 🤔"

Se on loistava ajatus! Katsos, kun resepti on määritelty, voit arvioida tarvittavat parametrit datan esikäsittelyä varten ja sitten poimia esikäsitellyn datan. Tätä ei yleensä tarvitse tehdä, kun käytät Tidymodels-kirjastoa (näemme normaalin käytännön aivan pian -> `workflows`), mutta siitä voi olla hyötyä, kun haluat tehdä jonkinlaisen järkevyystarkistuksen varmistaaksesi, että reseptit toimivat odotetusti.

Tätä varten tarvitset kaksi lisäverbiä: `prep()` ja `bake()`, ja kuten aina, pienet R-ystävämme [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) auttavat sinua ymmärtämään tämän paremmin!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Taiteilija: @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): arvioi tarvittavat parametrit harjoitusaineistosta, joita voidaan myöhemmin soveltaa muihin aineistoihin. Esimerkiksi, tietylle ennustajan sarakkeelle, mikä havainto saa kokonaisluvun 0, 1, 2 jne.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): ottaa valmiiksi käsitellyn reseptin ja soveltaa toimenpiteet mihin tahansa aineistoon.

Sanottuani tämän, valmistellaan ja sovelletaan reseptimme varmistaaksemme, että pinnan alla ennustajan sarakkeet koodataan ensin ennen mallin sovittamista.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 Käsitelty data `baked_pumpkins` sisältää kaikki ennustemuuttujat koodattuina, mikä vahvistaa, että määritellyt esikäsittelyvaiheet toimivat odotetusti. Tämä tekee datan lukemisesta vaikeampaa sinulle, mutta paljon ymmärrettävämpää Tidymodelsille! Käytä hetki aikaa selvittääksesi, mikä havainto on yhdistetty vastaavaan kokonaislukuun.

On myös syytä mainita, että `baked_pumpkins` on data frame, jonka avulla voimme suorittaa laskutoimituksia.

Esimerkiksi, yritetään löytää hyvä korrelaatio kahden datasi pisteen välillä, jotta voimme mahdollisesti rakentaa hyvän ennustemallin. Käytämme tähän funktiota `cor()`. Kirjoita `?cor()` saadaksesi lisätietoa funktiosta.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Kuten käy ilmi, kaupungin ja hinnan välillä on vain heikko korrelaatio. Sen sijaan paketin ja sen hinnan välillä on hieman parempi korrelaatio. Tämä on järkevää, eikö? Yleensä mitä suurempi tuotekuori, sitä korkeampi hinta.

Samalla voimme myös yrittää visualisoida kaikkien sarakkeiden korrelaatiomatriisin käyttämällä `corrplot`-pakettia.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Paljon parempi.

Hyvä kysymys, jonka voimme nyt esittää tästä datasta, on: '`Mitä hintaa voin odottaa tietylle kurpitsapaketille?`' Mennään suoraan asiaan!

> Huomio: Kun **`bake()`**-funktiolla käsittelet valmisteltua reseptiä **`pumpkins_prep`** käyttäen **`new_data = NULL`**, saat käsitellyn (eli koodatun) harjoitusdatan. Jos sinulla olisi toinen datasetti, esimerkiksi testidatasetti, ja haluaisit nähdä, miten resepti esikäsittelisi sen, voisit yksinkertaisesti käsitellä **`pumpkins_prep`**-reseptiä käyttäen **`new_data = test_set`**.

## 4. Rakenna lineaarinen regressiomalli

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografiikka: Dasani Madipalli</figcaption>


Nyt kun olemme rakentaneet reseptin ja vahvistaneet, että data esikäsitellään asianmukaisesti, rakennetaan nyt regressiomalli vastataksemme kysymykseen: `Mitä hintaa voin odottaa tietylle kurpitsapakkaukselle?`

#### Koulutetaan lineaarinen regressiomalli käyttämällä harjoitusjoukkoa

Kuten olet ehkä jo huomannut, sarake *price* on `tulosmuuttuja`, kun taas sarake *package* on `ennustemuuttuja`.

Tätä varten jaamme ensin datan siten, että 80 % menee harjoitusjoukkoon ja 20 % testijoukkoon, määrittelemme reseptin, joka koodaa ennustemuuttujan sarakkeen kokonaislukujen joukkoon, ja rakennamme mallin spesifikaation. Emme valmistele ja paista reseptiämme, koska tiedämme jo, että se esikäsittelee datan odotetulla tavalla.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Hyvää työtä! Nyt kun meillä on resepti ja mallin määrittely, meidän täytyy löytää tapa yhdistää ne yhdeksi objektiksi, joka ensin esikäsittelee datan (prep+bake kulissien takana), sovittaa mallin esikäsiteltyyn dataan ja mahdollistaa myös mahdolliset jälkikäsittelytoimet. Kuulostaa hyvältä, eikö vain!🤩

Tidymodels-paketissa tämä kätevä objekti tunnetaan nimellä [`workflow`](https://workflows.tidymodels.org/), ja se pitää mallikomponenttisi kätevästi yhdessä! Tätä kutsuttaisiin *pipelines*-nimellä *Pythonissa*.

Joten paketoidaan kaikki yhteen workflow-objektiin!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Kaupan päälle, työnkulku voidaan sovittaa/kouluttaa samalla tavalla kuin malli.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Mallin tuloksista voimme nähdä koulutuksen aikana opitut kertoimet. Ne edustavat parhaiten sovitetun suoran kertoimia, joka antaa meille pienimmän kokonaisvirheen todellisen ja ennustetun muuttujan välillä.

#### Mallin suorituskyvyn arviointi testijoukon avulla

On aika nähdä, miten malli suoriutui 📏! Miten tämä tehdään?

Nyt kun olemme kouluttaneet mallin, voimme käyttää sitä tekemään ennusteita testijoukolle `parsnip::predict()`-funktiolla. Sen jälkeen voimme verrata näitä ennusteita todellisiin arvoihin arvioidaksemme, kuinka hyvin (tai huonosti!) malli toimii.

Aloitetaan tekemällä ennusteita testijoukolle ja liitetään sitten sarakkeet testijoukkoon.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Kyllä, olet juuri kouluttanut mallin ja käyttänyt sitä ennusteiden tekemiseen!🔮 Onko se hyvä? Arvioidaan mallin suorituskykyä!

Tidymodels-paketissa tämä tehdään `yardstick::metrics()`-funktiolla! Lineaarisessa regressiossa keskitytään seuraaviin mittareihin:

-   `Root Mean Square Error (RMSE)`: [MSE](https://en.wikipedia.org/wiki/Mean_squared_error)-arvon neliöjuuri. Tämä antaa absoluuttisen mittarin samassa yksikössä kuin kohde (tässä tapauksessa kurpitsan hinta). Mitä pienempi arvo, sitä parempi malli (yksinkertaistetusti tämä edustaa keskimääräistä hintaa, jolla ennusteet ovat väärässä!)

-   `Coefficient of Determination (yleisesti tunnettu nimellä R-squared tai R2)`: Suhteellinen mittari, jossa suurempi arvo tarkoittaa parempaa mallin sovitusta. Käytännössä tämä mittari kuvaa, kuinka paljon mallin ennusteiden ja todellisten kohdearvojen välinen vaihtelu pystytään selittämään.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Siinä meni mallin suorituskyky. Katsotaan, voimmeko saada paremman käsityksen visualisoimalla hajontakaavion paketin ja hinnan välillä ja käyttämällä ennusteita parhaan sovituslinjan piirtämiseen.

Tämä tarkoittaa, että meidän täytyy valmistella ja käsitellä testijoukko koodataksemme pakettisarakkeen ja yhdistääksemme tämän mallimme tekemien ennusteiden kanssa.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Hienoa! Kuten huomaat, lineaarinen regressiomalli ei kovin hyvin yleistä suhdetta paketin ja sen vastaavan hinnan välillä.

🎃 Onnittelut, loit juuri mallin, joka voi auttaa ennustamaan muutamien kurpitsalajikkeiden hinnan. Kurpitsatilasi tulee olemaan upea tänä juhlapyhänä. Mutta luultavasti voit luoda vielä paremman mallin!

## 5. Rakenna polynomiregressiomalli

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografiikka: Dasani Madipalli</figcaption>


<!--![Infografiikka: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Joskus datamme ei välttämättä noudata lineaarista suhdetta, mutta haluamme silti ennustaa lopputuloksen. Polynominen regressio voi auttaa meitä tekemään ennusteita monimutkaisemmille epälineaarisille suhteille.

Otetaan esimerkiksi suhde paketin ja hinnan välillä kurpitsadatasarjassamme. Vaikka muuttujien välillä joskus on lineaarinen suhde - mitä suurempi kurpitsa tilavuudeltaan, sitä korkeampi hinta - näitä suhteita ei aina voida kuvata tasolla tai suoralla viivalla.

> ✅ Tässä on [joitakin esimerkkejä](https://online.stat.psu.edu/stat501/lesson/9/9.8) datasta, joka voisi hyötyä polynomisesta regressiosta
>
> Katsotaanpa uudelleen suhdetta lajikkeen ja hinnan välillä edellisessä kaaviossa. Vaikuttaako tämä hajontakaavio siltä, että sitä pitäisi välttämättä analysoida suoralla viivalla? Ehkä ei. Tässä tapauksessa voit kokeilla polynomista regressiota.
>
> ✅ Polynomit ovat matemaattisia lausekkeita, jotka voivat koostua yhdestä tai useammasta muuttujasta ja kertoimesta

#### Kouluta polynominen regressiomalli käyttämällä harjoitusjoukkoa

Polynominen regressio luo *kaarevan viivan*, joka sopii paremmin epälineaariseen dataan.

Katsotaan, suoriutuuko polynominen malli paremmin ennusteiden tekemisessä. Noudatamme melko samanlaista menettelyä kuin aiemmin:

-   Luo resepti, joka määrittää esikäsittelyvaiheet, jotka datalle tulee suorittaa, jotta se on valmis mallinnukseen, kuten: ennustajien koodaus ja polynomien laskeminen asteelle *n*

-   Rakenna mallin spesifikaatio

-   Yhdistä resepti ja mallin spesifikaatio työnkulkuun

-   Luo malli sovittamalla työnkulku

-   Arvioi, kuinka hyvin malli suoriutuu testidatalla

Aloitetaan heti!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Arvioi mallin suorituskyky

👏👏Olet rakentanut polynomimallin, tehdään ennusteita testijoukolla!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, arvioidaan kuinka malli suoriutui testijoukosta käyttämällä `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Paljon parempi suorituskyky.

`rmse` pieneni noin 7:stä noin 3:een, mikä osoittaa, että todellisen hinnan ja ennustetun hinnan välinen virhe on vähentynyt. Voit *vapaasti* tulkita tämän tarkoittavan, että keskimäärin väärät ennusteet ovat noin 3 dollaria pielessä. `rsq` nousi noin 0.4:stä 0.8:aan.

Kaikki nämä mittarit osoittavat, että polynomimalli toimii huomattavasti paremmin kuin lineaarinen malli. Hyvää työtä!

Katsotaanpa, voimmeko visualisoida tämän!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Voit nähdä kaarevan viivan, joka sopii paremmin dataasi! 🤩

Voit tehdä tästä vielä tasaisemman antamalla polynomikaavan `geom_smooth`-funktiolle näin:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Aivan kuin tasainen käyrä!🤩

Näin voit tehdä uuden ennusteen:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


`polynomial model` -ennuste vaikuttaa järkevältä, kun tarkastellaan `price` ja `package` hajontakaavioita! Ja jos tämä malli on parempi kuin edellinen, samoja tietoja tarkasteltaessa, sinun täytyy budjetoida näitä kalliimpia kurpitsoja varten!

🏆 Hyvin tehty! Loit kaksi regressiomallia yhden oppitunnin aikana. Regressioon liittyvän viimeisen osion aikana opit logistisesta regressiosta kategorioiden määrittämiseksi.

## **🚀Haaste**

Testaa useita eri muuttujia tässä muistikirjassa ja katso, miten korrelaatio vaikuttaa mallin tarkkuuteen.

## [**Oppitunnin jälkeinen kysely**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Kertaus & Itseopiskelu**

Tässä oppitunnissa opimme lineaarisesta regressiosta. On olemassa myös muita tärkeitä regressiotyyppejä. Lue Stepwise-, Ridge-, Lasso- ja Elasticnet-tekniikoista. Hyvä kurssi lisäopiskeluun on [Stanford Statistical Learning -kurssi](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Jos haluat oppia lisää siitä, miten käyttää upeaa Tidymodels-kehystä, tutustu seuraaviin resursseihin:

-   Tidymodels-verkkosivusto: [Aloita Tidymodelsilla](https://www.tidymodels.org/start/)

-   Max Kuhn ja Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **KIITOS:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) upeiden kuvitusten luomisesta, jotka tekevät R:stä kutsuvamman ja kiinnostavamman. Löydät lisää kuvituksia hänen [galleriastaan](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
